# Introduction to NDVI with Google Earth Engine

This notebook introduces the **Normalized Difference Vegetation Index (NDVI)** and demonstrates how to calculate it using satellite imagery from Google Earth Engine.

## Learning Objectives
- Understand what NDVI measures and why it's useful
- Learn to work with Landsat and Sentinel-2 imagery
- Calculate NDVI manually and using the platform utilities
- Visualize vegetation health with interactive maps

## Prerequisites
- Google Earth Engine account
- Python environment with required packages installed

## 1. What is NDVI?

**NDVI (Normalized Difference Vegetation Index)** is a standardized index that measures vegetation greenness based on how plants reflect light at different wavelengths.

### The Science Behind NDVI

Plants absorb visible light (especially red) for photosynthesis and strongly reflect near-infrared (NIR) light. Healthy vegetation has:
- **Low red reflectance** (absorbed for photosynthesis)
- **High NIR reflectance** (reflected by cell structure)

### The Formula

$$NDVI = \frac{NIR - Red}{NIR + Red}$$

### NDVI Value Interpretation

| NDVI Range | Interpretation |
|------------|----------------|
| -1.0 to 0.0 | Water, clouds, snow |
| 0.0 to 0.2 | Bare soil, rock, urban areas |
| 0.2 to 0.4 | Sparse vegetation, grassland |
| 0.4 to 0.6 | Moderate vegetation |
| 0.6 to 0.8 | Dense vegetation |
| 0.8 to 1.0 | Very dense, healthy vegetation |

## 2. Setup and Authentication

In [ ]:
# Install required packages (uncomment if needed)
# !pip install earthengine-api geemap folium

In [ ]:
import ee
import folium

# Initialize Earth Engine
# First time: run `earthengine authenticate` in terminal
try:
    ee.Initialize()
    print("Earth Engine initialized successfully!")
except Exception as e:
    print(f"Error initializing Earth Engine: {e}")
    print("Run 'earthengine authenticate' in your terminal first.")

## 3. Define an Area of Interest

Let's define a study area. We'll use the Amazon rainforest as an example.

In [ ]:
# Define a bounding box in the Amazon rainforest
# Format: [min_lon, min_lat, max_lon, max_lat]
amazon_bbox = [-62.5, -4.0, -62.0, -3.5]

# Create an Earth Engine geometry
aoi = ee.Geometry.Rectangle(amazon_bbox)

# Get center for map
center_lat = (amazon_bbox[1] + amazon_bbox[3]) / 2
center_lon = (amazon_bbox[0] + amazon_bbox[2]) / 2

print(f"Study area center: {center_lat:.2f}°N, {center_lon:.2f}°E")
print(f"Bounding box: {amazon_bbox}")

## 4. Load Landsat 8 Imagery

Landsat 8 provides free satellite imagery with 30m resolution since 2013.

In [ ]:
# Load Landsat 8 Collection 2 Surface Reflectance
landsat8 = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterBounds(aoi)
    .filterDate('2023-01-01', '2023-12-31')
    .filter(ee.Filter.lt('CLOUD_COVER', 20))
)

print(f"Found {landsat8.size().getInfo()} Landsat 8 images")

In [ ]:
# Landsat 8 band names
# SR_B4 = Red (0.64-0.67 μm)
# SR_B5 = NIR (0.85-0.88 μm)

# Scale factors for Collection 2 Level-2
SCALE_FACTOR = 0.0000275
OFFSET = -0.2

def scale_landsat(image):
    """Apply scale factors to Landsat Collection 2 imagery."""
    optical = image.select('SR_B.').multiply(SCALE_FACTOR).add(OFFSET)
    return image.addBands(optical, overwrite=True)

# Apply scaling and create median composite
composite = landsat8.map(scale_landsat).median().clip(aoi)
print("Created median composite")

## 5. Calculate NDVI Manually

Let's calculate NDVI step by step to understand the process.

In [ ]:
# Extract NIR and Red bands
nir = composite.select('SR_B5')  # NIR band
red = composite.select('SR_B4')  # Red band

# Calculate NDVI manually
ndvi_manual = nir.subtract(red).divide(nir.add(red)).rename('ndvi')

print("NDVI calculated manually!")
print("Formula: (NIR - Red) / (NIR + Red)")

In [ ]:
# Earth Engine has a built-in function for normalized difference
ndvi_builtin = composite.normalizedDifference(['SR_B5', 'SR_B4']).rename('ndvi')

print("NDVI calculated with normalizedDifference()!")

## 6. Using the Platform Utilities

Our platform provides convenient functions for index calculation.

In [ ]:
# Import from our engine module
from engine.indices import add_ndvi, add_all_indices
from engine.composites import create_fused_composite, harmonize_bands
from engine.config import TEMPORAL_PERIODS

# The platform harmonizes band names across sensors
# After harmonization: 'nir', 'red', 'green', 'blue', 'swir1', 'swir2'

print("Platform modules imported successfully!")

In [ ]:
# Create a composite using the platform
period_info = TEMPORAL_PERIODS['present']

platform_composite = create_fused_composite(
    aoi=aoi,
    start_date=period_info['start'],
    end_date=period_info['end'],
    sensors=period_info['sensors'],
    cloud_threshold=20.0,
)

# Add NDVI using platform function
platform_composite = add_ndvi(platform_composite)

print("Created platform composite with NDVI!")
print(f"Bands: {platform_composite.bandNames().getInfo()}")

## 7. Visualize NDVI on a Map

Let's create an interactive map to visualize our NDVI results.

In [ ]:
# Define visualization parameters for NDVI
ndvi_vis_params = {
    'min': -0.2,
    'max': 0.8,
    'palette': [
        '#d73027',  # Red - bare soil/water
        '#fc8d59',  # Orange - sparse vegetation
        '#fee08b',  # Yellow - low vegetation
        '#d9ef8b',  # Light green - moderate
        '#91cf60',  # Green - dense vegetation
        '#1a9850',  # Dark green - very dense
    ]
}

# RGB visualization for reference
rgb_vis_params = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0,
    'max': 0.3,
    'gamma': 1.4,
}

In [ ]:
# Create a folium map
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=11,
    tiles='OpenStreetMap'
)

# Add RGB layer
rgb_map_id = composite.getMapId(rgb_vis_params)
folium.TileLayer(
    tiles=rgb_map_id['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    name='RGB Composite',
    overlay=True,
    control=True,
).add_to(m)

# Add NDVI layer
ndvi_map_id = ndvi_manual.getMapId(ndvi_vis_params)
folium.TileLayer(
    tiles=ndvi_map_id['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    name='NDVI',
    overlay=True,
    control=True,
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Display the map
m

## 8. Calculate NDVI Statistics

In [ ]:
# Calculate statistics within our AOI
stats = ndvi_manual.reduceRegion(
    reducer=ee.Reducer.mean()
        .combine(ee.Reducer.min(), '', True)
        .combine(ee.Reducer.max(), '', True)
        .combine(ee.Reducer.stdDev(), '', True),
    geometry=aoi,
    scale=30,
    maxPixels=1e9,
).getInfo()

print("NDVI Statistics for Study Area")
print("=" * 35)
print(f"Mean:   {stats['ndvi_mean']:.3f}")
print(f"Min:    {stats['ndvi_min']:.3f}")
print(f"Max:    {stats['ndvi_max']:.3f}")
print(f"StdDev: {stats['ndvi_stdDev']:.3f}")

## 9. NDVI Histogram

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get histogram data from Earth Engine
histogram = ndvi_manual.reduceRegion(
    reducer=ee.Reducer.histogram(100),
    geometry=aoi,
    scale=30,
    maxPixels=1e9,
).getInfo()

# Extract histogram values
hist_data = histogram['ndvi']
bucket_means = hist_data['bucketMeans']
counts = hist_data['histogram']

# Plot histogram
plt.figure(figsize=(10, 6))
plt.bar(bucket_means, counts, width=0.01, color='green', alpha=0.7)
plt.xlabel('NDVI Value')
plt.ylabel('Pixel Count')
plt.title('NDVI Distribution in Study Area')
plt.axvline(x=0.2, color='orange', linestyle='--', label='Sparse vegetation threshold')
plt.axvline(x=0.4, color='green', linestyle='--', label='Moderate vegetation threshold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 10. Export NDVI as GeoTIFF

Export the NDVI image to Google Drive for use in GIS software.

In [ ]:
# Create export task
task = ee.batch.Export.image.toDrive(
    image=ndvi_manual,
    description='NDVI_Amazon_2023',
    folder='GEE_Exports',
    fileNamePrefix='ndvi_amazon_2023',
    region=aoi,
    scale=30,
    crs='EPSG:4326',
    maxPixels=1e13,
    formatOptions={'cloudOptimized': True},
)

# Uncomment to start the export
# task.start()
# print(f"Export started! Check Google Drive folder 'GEE_Exports'")
# print(f"Task ID: {task.id}")

print("Export task created (not started - uncomment task.start() to run)")

## Exercises

1. **Try a different location**: Modify the `amazon_bbox` to analyze a different region (e.g., your local area).

2. **Compare seasons**: Create two composites (wet and dry season) and compare NDVI values.

3. **Add other indices**: Use `add_all_indices()` to calculate NBR, NDWI, and EVI alongside NDVI.

4. **Classify vegetation**: Create a classified image with categories like:
   - Water (NDVI < 0)
   - Bare soil (0 < NDVI < 0.2)
   - Sparse vegetation (0.2 < NDVI < 0.4)
   - Dense vegetation (NDVI > 0.4)

## Summary

In this notebook, we learned:

1. **NDVI fundamentals**: How plants reflect light and why NDVI is useful
2. **Earth Engine basics**: Loading imagery, filtering, and creating composites
3. **Index calculation**: Both manual and using platform utilities
4. **Visualization**: Creating interactive maps with Folium
5. **Statistics**: Calculating mean, min, max, and histograms
6. **Export**: Saving results as GeoTIFF files

## Next Steps

Continue to **002_change_detection.ipynb** to learn how to detect vegetation changes over time!